Data and Project source: https://www.kaggle.com/competitions/gan-getting-started/overview

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import os
import torchvision.transforms as transforms
import torch.optim as optim
from tqdm import tqdm

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
photo_path = 'picashu/data/photo_jpg'
monet_path = 'picashu/data/monet_jpg'

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [ ]:
# Dataset class:

class Images(Dataset):
    def __init__(self, photo_path, monet_path, transform):
        self.photo_path = photo_path
        self.monet_path = monet_path
        self.transform = transform
        self.photos = os.listdir(photo_path)
        self.monets = os.listdir(monet_path)
        self.l_photo = len(self.photos)
        self.l_monet = len(self.monets)
    
    def __len__(self):
        return max(len(self.photos), len(self.monets))
    
    def __getitem__(self, idx):
        photo = Image.open(self.photo_path + self.photos[idx % self.l_photo]).convert("RGB")
        monet = Image.open(self.monet_path + self.monets[idx % self.l_monet]).convert("RGB")
        
        photo = self.transform(photo)
        monet = self.transform(monet)
        
        return photo, monet

In [ ]:
# Define dataset:

dataset = Images(photo_path, monet_path, transform)

In [ ]:
# Define torch dataloader:

dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [ ]:
# Check examples of pics:
example = next(iter(dataloader))

plt.subplot(1, 2, 1)
plt.title('Photo example')
plt.imshow(example[0][0].permute(1, 2, 0) * 0.5 + 0.5)

plt.subplot(1, 2, 2)
plt.title('Monet example')
plt.imshow(example[1][0].permute(1, 2, 0) * 0.5 + 0.5)

Discriminator model

1. conv_block Class:

This class defines a reusable convolutional block.

Purpose: To create a series of convolutional layers followed by normalization and activation, which are commonly used in GANs to downsample the input image and extract features.

Components:
__init__ Method:

nn.Conv2d: Applies a 2D convolution to the input image. The parameters include:
in_channels: Number of input channels (e.g., 3 for RGB images).
out_channels: Number of filters used in the convolution, which determines the output channels.
kernel_size=4: A 4x4 filter size.
stride: Controls the step size of the convolution. A stride of 2 halves the image size, while 1 retains it.
padding=1: Adds padding to the input to maintain the image's spatial size.
padding_mode="reflect": Uses reflection padding to avoid boundary artifacts.
bias=True: Adds a bias term.
nn.InstanceNorm2d: Normalizes the activations across each channel to stabilize training.
nn.LeakyReLU(0.2): A leaky ReLU activation function with a slope of 0.2 for negative values, allowing a small gradient to pass through.

forward Method:

This method defines how data flows through the layers. It takes an input tensor x and applies the sequential operations defined in __init__.

In [ ]:
class conv_block(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=in_channels,
                      out_channels=out_channels, 
                      kernel_size=4, 
                      stride=stride,
                      padding=1,
                      bias=True,
                      padding_mode="reflect"),
            nn.InstanceNorm2d(out_channels),
            nn.LeakyReLU(0.2)
        )
    
    def forward(self, x):
        return self.conv(x)

2. Discriminator Class:

This class defines the architecture of the discriminator model, which classifies whether an input image is real or fake.

Purpose: In GANs, the discriminator learns to distinguish real images from fake ones generated by the generator.


Components:
__init__ Method:

self.initial: The first layer of the discriminator:
nn.Conv2d: A 2D convolution similar to the one in conv_block, with in_channels=3 for RGB images, out_channels=64, and a stride of 2 to downsample the image.
nn.LeakyReLU(0.2): Activation after the initial convolution.
self.process: The main sequence of layers after the initial one:
Several conv_blocks, each progressively downsampling and increasing the number of filters:
conv_block(64, 128, 2): From 64 to 128 channels, halving the spatial dimensions.
conv_block(128, 256, 2): From 128 to 256 channels.
conv_block(256, 512, 1): From 256 to 512 channels but with no spatial reduction due to stride=1.
A final convolution layer nn.Conv2d(512, 1, kernel_size=4, stride=1) to reduce the feature map to a single output channel.
nn.Sigmoid(): Outputs a probability value between 0 and 1 (fake or real).

forward Method:

This method defines the forward pass of the discriminator.

The input tensor passes through the initial layer and then through the layers in process. The final output is a tensor that indicates whether the input is real or fake.


Output Shape:
The input is assumed to have the shape [batch_size, 3, 256, 256] (e.g., a batch of RGB images with a resolution of 256x256).
The output tensor has a shape of [batch_size, 1, 30, 30], meaning that the model generates a 30x30 grid of "real/fake" predictions for different patches of the input image.

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, in_channels=3):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=64,
                kernel_size=4, 
                stride=2,
                padding=1,
                padding_mode="reflect"),
            nn.LeakyReLU(0.2)
        )
        self.process = nn.Sequential(
            conv_block(64, 128, 2),
            conv_block(128, 256, 2),
            conv_block(256, 512, 1),
            nn.Conv2d(
                in_channels=512,
                out_channels=1,
                kernel_size=4,
                stride=1,
                padding=1,
                padding_mode='reflect'),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        """
        OUT = floor((IN + 2 * padding - kernel_size + 1) / stride + 1)
        [batch_size, 3, 256, 256] ->
        [batch_size, 64, 128, 128] ->
        [batch_size, 128, 64, 64] ->
        [batch_size, 256, 32, 32] ->
        [batch_size, 512, 30, 30] ->
        [batch_size, 1, 30, 30]
        """
        x = self.initial(x)
        x = self.process(x)
        return x

In [ ]:
x = torch.randn((1, 3, 256, 256))
dis = Discriminator()
assert(dis(x).shape == (1, 1, 30, 30))

Generator Model

1. gen_conv_block Class:

This class defines a convolutional block used for both downsampling (convolution) and upsampling (transpose convolution).

Components:

__init__ Method:

in_channels and out_channels: The number of input and output channels for the block.
TYPE: Determines whether the block performs downsampling ('down' for convolution) or upsampling ('up' for transposed convolution).
activation: A flag indicating whether to apply an activation function (ReLU) after normalization.
Downsampling: If TYPE == 'down', a nn.Conv2d layer is used to reduce the spatial dimensions of the input.
Upsampling: If TYPE == 'up', a nn.ConvTranspose2d layer is used to increase the spatial dimensions.
Instance Normalization: nn.InstanceNorm2d(out_channels) normalizes the activations across each channel.
Activation: nn.ReLU(inplace=True) if activation=True, otherwise nn.Identity(), which means no activation.


forward Method:
Defines the forward pass, applying the convolution or transposed convolution, normalization, and optionally the activation function.

In [ ]:
class gen_conv_block(nn.Module):
    def __init__(self, in_channels, out_channels, TYPE='down', activation=False, **kwargs):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=in_channels,
                      out_channels=out_channels,
                      padding_mode="reflect",
                      **kwargs) if TYPE == 'down'
            else nn.ConvTranspose2d(in_channels=in_channels,
                                   out_channels=out_channels,
                                   **kwargs),
            nn.InstanceNorm2d(out_channels),
            nn.ReLU(inplace=True) if activation else nn.Identity()
        )
        
    def forward(self, x):
        return self.conv(x)

2. res_block Class:
This class defines a residual block, which is commonly used in ResNet architectures. Residual blocks help with training deep networks by allowing gradients to pass through more easily.

Components:
__init__ Method:
Two convolutional blocks (using gen_conv_block) with the same number of channels. These blocks operate with a kernel_size=3 and padding=1 to preserve spatial dimensions.
The second block does not use an activation function (activation=False).

forward Method:
Applies the residual connection: x + self.block_(x). This adds the input x to the output of the two convolutional layers (self.block_), allowing gradients to propagate more easily through the network.

In [ ]:
class res_block(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.block_ = nn.Sequential(
            gen_conv_block(channels, channels, kernel_size=3, padding=1),
            gen_conv_block(channels, channels, activation=False, kernel_size=3, padding=1)
        )
    
    def forward(self, x):
        return x + self.block_(x)

3. Generator Class:
This class defines the Generator architecture, responsible for producing an output image from an input image.

Components:

__init__ Method:
Initial Convolution:
nn.Conv2d(in_channels, 64, kernel_size=7, stride=1, padding=3, padding_mode="reflect"): A 7x7 convolution that preserves spatial dimensions (thanks to padding=3) and increases the number of channels from in_channels (usually 3 for RGB images) to 64.
nn.ReLU(inplace=True): Applies ReLU activation.

Downsampling:
self.down: This uses two gen_conv_blocks to downsample the input image while increasing the number of channels. The spatial dimensions are halved at each step (stride=2), while the number of channels increases:
First gen_conv_block(64, 64*2): Reduces spatial dimensions and increases channels to 128.
Second gen_conv_block(64*2, 64*4): Reduces dimensions further and increases channels to 256.

Residual Blocks:
self.residual: This contains a sequence of residual blocks (the res_block class). The number of residual blocks is defined by num_residuals_blocks, which is 9 in this case.
Each res_block keeps the spatial dimensions and number of channels constant (256 channels).


Upsampling:
self.up: This contains two gen_conv_blocks that perform upsampling (with TYPE='up'):
First gen_conv_block(64*4, 64*2): Doubles the spatial dimensions and reduces the number of channels from 256 to 128.
Second gen_conv_block(64*2, 64): Doubles the spatial dimensions again and reduces the number of channels from 128 to 64.


Final Image Generation:
self.get_img: This final convolution converts the output of the upsampling blocks back to the original number of channels (in_channels, usually 3 for RGB images).
The kernel size of 7x7 is used again, and padding=3 ensures that the spatial dimensions of the output match the input.


forward Method:
The input x goes through the following stages:
Initial Convolution (self.initial).
Downsampling (self.down): Reduces spatial dimensions and increases channels.
Residual Blocks (self.residual): Processes the features using 9 residual blocks.
Upsampling (self.up): Increases spatial dimensions and decreases channels.
Image Generation (self.get_img): Converts the processed feature map back into an image with the original number of channels.



Summary of the Flow:
Input Image → Initial convolution → Downsampling → Residual blocks → Upsampling → Generated Image.


In [ ]:
class Generator(nn.Module):
    def __init__(self, in_channels=3, num_residuals_blocks=9):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=7, stride=1, padding=3, padding_mode="reflect"),
            nn.ReLU(inplace=True)
        )
        self.down = nn.Sequential(
            gen_conv_block(64, 64*2, TYPE='down', kernel_size=3, stride=2, padding=1),
            gen_conv_block(64*2, 64*4, TYPE='down', kernel_size=3, stride=2, padding=1)
        )
        self.residual = nn.Sequential(
            *[res_block(64*4) for _ in range(num_residuals_blocks)]
        )
        self.up = nn.Sequential(
            gen_conv_block(64*4, 64*2, TYPE='up', kernel_size=3, stride=2, padding=1, output_padding=1),
            gen_conv_block(64*2, 64, TYPE='up', kernel_size=3, stride=2, padding=1, output_padding=1)
        )
        self.get_img = nn.Conv2d(64, in_channels, kernel_size=7, stride=1, padding=3, padding_mode="reflect")
        
    def forward(self, x):
        x = self.initial(x)
        x = self.down(x)
        x = self.residual(x)
        x = self.up(x)
        return self.get_img(x)

In [ ]:
x = torch.randn((1, 3, 256, 256))
dis = Generator()
assert(dis(x).shape == (1, 3, 256, 256))

In [ ]:
lr = 2e-4
lambda_cycle = 10

In [ ]:
disc_photo = Discriminator().to(device)
disc_monet = Discriminator().to(device)

gen_photo = Generator().to(device)
gen_monet = Generator().to(device)

In [ ]:
disc_optimizer = optim.Adam(
    list(disc_photo.parameters()) + list(disc_monet.parameters()),
    lr=lr,
    betas=(0.5, 0.999)
)

gen_optimizer = optim.Adam(
    list(gen_photo.parameters()) + list(gen_monet.parameters()),
    lr=lr,
    betas=(0.5, 0.999)
)

In [ ]:
dis_scaler = torch.amp.GradScaler('cuda')
gen_scaler = torch.amp.GradScaler('cuda')

In [ ]:
MSE = nn.MSELoss()
L1 = nn.L1Loss()

Training Models

In [ ]:
epoches = 1

for epoch in range(epoches):
    running_dis_loss = 0.0
    running_gen_loss = 0.0
    for photo, monet in tqdm(dataloader, leave=True):
        photo = photo.to(device)
        monet = monet.to(device)
        
        # Train discriminator:
        fake_photo = gen_photo(monet)
        Dis_photo_real = disc_photo(photo)
        Dis_photo_fake = disc_photo(fake_photo.detach())
        
        Dis_photo_loss = MSE(Dis_photo_real, torch.ones_like(Dis_photo_real)) + \
                         MSE(Dis_photo_fake, torch.zeros_like(Dis_photo_fake))
        
        fake_monet = gen_monet(photo)
        Dis_monet_real = disc_monet(monet)
        Dis_monet_fake = disc_monet(fake_monet.detach())
        
        Dis_monet_loss = MSE(Dis_monet_real, torch.ones_like(Dis_monet_real)) + \
                         MSE(Dis_monet_fake, torch.zeros_like(Dis_monet_fake))
        
        Dis_loss = (Dis_photo_loss + Dis_monet_loss) / 2.0
        running_dis_loss += Dis_loss / len(dataloader)
        
        disc_optimizer.zero_grad()
        dis_scaler.scale(Dis_loss).backward()
        dis_scaler.step(disc_optimizer)
        dis_scaler.update()
        
        # Train Generator:
        Dis_photo_fake = disc_photo(fake_photo)
        Dis_monet_fake = disc_monet(fake_monet)
        
        Gen_photo_loss = MSE(Dis_photo_fake, torch.ones_like(Dis_photo_fake))
        Gen_monet_loss = MSE(Dis_monet_fake, torch.ones_like(Dis_monet_fake))
        
        Cycled_monet = gen_monet(fake_photo) 
        Cycled_photo = gen_photo(fake_monet)
        
        Cycled_loss = L1(monet, Cycled_monet) + L1(photo, Cycled_photo)
        
        Gen_loss = Gen_photo_loss + Gen_monet_loss + Cycled_loss * lambda_cycle
        running_gen_loss += Gen_loss / len(dataloader)
        
        gen_optimizer.zero_grad()
        gen_scaler.scale(Gen_loss).backward()
        gen_scaler.step(gen_optimizer)
        gen_scaler.update()
    print(f"Epoch {epoch + 1}. Generator loss by epoch: {running_gen_loss}, discriminator loss by epoch: {running_dis_loss}")

In [ ]:
torch.save(disc_photo.state_dict(), '/kaggle/working/disc_photo.pth')
torch.save(disc_monet.state_dict(), '/kaggle/working/disc_monet.pth')
torch.save(gen_photo.state_dict(), '/kaggle/working/gen_photo.pth')
torch.save(gen_monet.state_dict(), '/kaggle/working/gen_monet.pth')

In [ ]:
batch = next(iter(dataloader))[0]

_, ax = plt.subplots(5, 2, figsize=(12, 12))

for i in range(5):
    original_img = batch[i]
    predicted_img = None
    with torch.no_grad():
        predicted_img = gen_monet(original_img.unsqueeze(0).to(device))
    
    ax[i, 0].imshow(original_img.permute(1, 2, 0) * 0.5 + 0.5)
    ax[i, 1].imshow(predicted_img.squeeze(0).permute(1, 2, 0).cpu() * 0.5 + 0.5)
    
    ax[i, 0].set_title("Original photo")
    ax[i, 1].set_title("Monet like")
    
    ax[i, 0].axis("off")
    ax[i, 1].axis("off")
plt.show()

This project is inspired from https://github.com/junyanz/CycleGAN